In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!python '/content/drive/MyDrive/My Drive/Colab Notebooks/Hypergraph_my/EDA.py'

In [ ]:
!pip install karateclub
!pip install node2vec
!pip install dgl

In [4]:
import warnings
warnings.filterwarnings('ignore')
import numba
from numba import jit,njit, cuda
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F
import itertools
import numpy as np
import scipy.sparse as sp
import networkx as nx
import pandas as pd
import tensorflow as tf
from sklearn.metrics import roc_auc_score, f1_score, recall_score,precision_score, accuracy_score,average_precision_score,precision_recall_curve,auc
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from karateclub import DeepWalk
from node2vec import Node2Vec

DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


In [7]:
'''
converting to sparse matrix
'''
df=LEN=645 # or 645 if you use TWOSIDES dataset
from scipy.sparse import coo_matrix
nl=coo_matrix((df, df))
nl.setdiag(1)
#nl.toarray()
values = nl.data
indices = np.vstack((nl.row, nl.col))
i = torch.LongTensor(indices)
v = torch.FloatTensor(values)
shape = nl.shape
nl=torch.sparse_coo_tensor(i, v, torch.Size(shape))

In [8]:
chemicalsub_drug = torch.load('/content/drive/MyDrive/My Drive/Colab Notebooks/Hypergraph_my/data/hyG_drug_645_kmer_3.pt')#reading the hypergraph for drugbank dataset for 9-mer. change it based on your choice
data_dict = {
        ('node', 'in', 'edge'): (chemicalsub_drug[:,0], chemicalsub_drug[:,1]),        
        ('edge', 'con', 'node'): (chemicalsub_drug[:,1], chemicalsub_drug[:,0])
    }

hyG = dgl.heterograph(data_dict)
n_chemicalsub=822 #change the num of rows based on the dataset. row info. is available on data folder
rows=n_chemicalsub
n_hedge=LEN
columns=n_hedge

drug_X=nl
hyG.ndata['h'] = {'edge' : torch.tensor(drug_X).type('torch.FloatTensor'), 'node' : torch.ones(rows, 128)}
e_feat = torch.tensor(drug_X).type('torch.FloatTensor')
v_feat = torch.ones(rows, 128)

In [9]:
hyG

Graph(num_nodes={'edge': 645, 'node': 822},
      num_edges={('edge', 'con', 'node'): 31631, ('node', 'in', 'edge'): 31631},
      metagraph=[('edge', 'node', 'con'), ('node', 'edge', 'in')])

In [10]:
#3:
src=[]
dst=[]

with open("/content/drive/MyDrive/My Drive/Colab Notebooks/Hypergraph_my/data/edge_list_regular_graph_645.txt") as fp:
        for i,line in enumerate (fp):
            info = line.strip().split()
            src.append(info[0])
            dst.append(info[1])

src=np.asarray(src, dtype=np.int64) #to use in the data prerpocessing
dst=np.asarray(dst, dtype=np.int64) #to use in the data prerpocessing

In [11]:
"""
Graph creation 
"""
from dgl.data import DGLDataset
import os
import torch.nn as nn
import torch.nn.functional as F

class KarateClubDataset(DGLDataset):
    def __init__(self):
        super().__init__(name='karate_club')
    def process(self):
        edges_src = torch.from_numpy(src)
        edges_dst = torch.from_numpy(dst)
        self.graph = dgl.graph((edges_src, edges_dst), num_nodes=LEN)
    def __getitem__(self, i):
        return self.graph
    def __len__(self):
        return 1

dataset = KarateClubDataset()
g = dataset[0]
print(g)

Graph(num_nodes=645, num_edges=126946,
      ndata_schemes={}
      edata_schemes={})


In [ ]:
'''
Graph creation
*************another approach ***********************************
'''
np.random.seed(42)
# Split edge set for training and testing
u, v = g.edges()
eids = np.arange(g.number_of_edges())
eids = np.random.permutation(eids)
eids=eids.tolist()
test_size = int(len(eids) * 0.1)
val_size = int(len(eids) * 0.1)
train_size=len(eids)-(test_size+val_size)
test_size1=int(test_size/2)


T1=[]
T2=[]
numba.njit(target="cuda")
def func2(T1):
  for i in eids:
    m, n = int(u[i]), int(v[i])
    c=(m,n)
    T1.append(c)
  return T1  
function1=jit(parallel=True) (func2)
T1 = function1(T1)


numba.njit(target="cuda")
def func3(T1,T2):
  for i in T1:
    m=i[0]
    n=i[1]
    c=(m,n)
    d=(n,m)
    if c not in T2 and d not in T2:
      T2.append(c)
    if len(T2)>=test_size1:
      break  
  return T2
function1=jit(parallel=True) (func3)
T2 = function1(T1,T2)


test_eids=[]
numba.njit(target="cuda")
def func4(T2,test_eids,g):
  for i in T2:
    m= i[0]
    n=i[1]
    c=g.edge_id(m,n)
    d=g.edge_id(n,m)
    test_eids.append(c)
    test_eids.append(d)
  return test_eids  
function1=jit(parallel=True) (func4)
test_eids = function1(T2,test_eids,g)

val_eids=[]
numba.njit(target="cuda")
def func5(eids,test_eids,val_eids):
  i=0
  for j in eids:
    a=int(eids[i])
    if a not in test_eids and len(val_eids)<val_size:
      val_eids.append(a)  
    i=i+1
  return val_eids    
function1=jit(parallel=True) (func5)
val_eids = function1(eids,test_eids,val_eids)     

train_eids=[]
numba.njit(target="cuda")
def func6(eids,test_eids,val_eids,train_eids):
  i=0
  for j in eids:
    a=int(eids[i])
    if a not in test_eids and a not in val_eids and len(train_eids)<train_size:
      train_eids.append(a)  
    i=i+1
  return train_eids    
function1=jit(parallel=True) (func6)
train_eids = function1(eids,test_eids,val_eids,train_eids)     

# main one. splitting into train and test set.
test_pos_u, test_pos_v = u[test_eids], v[test_eids]
train_pos_u, train_pos_v = u[train_eids], v[train_eids]
val_pos_u, val_pos_v = u[val_eids], v[val_eids]


# Find all negative edges and split them for training and testing
adj = sp.coo_matrix((np.ones(len(u)), (u.numpy(), v.numpy())))
adj_neg = 1 - adj.todense() - np.eye(g.number_of_nodes())
neg_u, neg_v = np.where(adj_neg != 0)

neg_eids = np.random.choice(len(neg_u), g.number_of_edges(),replace=False)
g_=nx.from_numpy_matrix(adj_neg)
g_=dgl.from_networkx(g_)

T1=[]
T2=[]
numba.njit(target="cuda")
def func2(T1):
  for i in neg_eids:
    m, n = int(neg_u[i]), int(neg_v[i])
    c=(m,n)
    T1.append(c)
  return T1  
function1=jit(parallel=True) (func2)
T1 = function1(T1)


numba.njit(target="cuda")
def func3(T1,T2):
  for i in T1:
    m=i[0]
    n=i[1]
    c=(m,n)
    d=(n,m)
    if c not in T2 and d not in T2:
      T2.append(c)
    if len(T2)>=test_size1:
      break  
  return T2
function1=jit(parallel=True) (func3)
T2 = function1(T1,T2)


test_neg_eids=[]
numba.njit(target="cuda")
def func4(T2,test_eids,g):
  for i in T2:
    m= i[0]
    n=i[1]
    c=g.edge_id(m,n)
    d=g.edge_id(n,m)
    test_neg_eids.append(c)
    test_neg_eids.append(d)
  return test_neg_eids  
function1=jit(parallel=True) (func4)
test_neg_eids = function1(T2,test_neg_eids,g_)

val_neg_eids=[]
numba.njit(target="cuda")
def func5(neg_eids,test_neg_eids,val_neg_eids):
  i=0
  for j in neg_eids:
    a=int(neg_eids[i])
    if a not in test_neg_eids and len(val_neg_eids)<(val_size):
      val_neg_eids.append(a)  
    i=i+1
  return val_neg_eids    
function1=jit(parallel=True) (func5)
val_neg_eids = function1(neg_eids,test_neg_eids,val_neg_eids)  

train_neg_eids=[]
numba.njit(target="cuda")
def func6(neg_eids,test_neg_eids,val_neg_eids,train_neg_eids):
  i=0
  for j in neg_eids:
    a=int(neg_eids[i])
    if a not in test_neg_eids and a not in val_neg_eids and len(train_neg_eids)<(train_size):
      train_neg_eids.append(a)  
    i=i+1
  return train_neg_eids    
function1=jit(parallel=True) (func6)
train_neg_eids = function1(neg_eids,test_neg_eids,val_neg_eids,train_neg_eids)     

test_neg_u, test_neg_v = neg_u[test_neg_eids], neg_v[test_neg_eids]
train_neg_u, train_neg_v = neg_u[train_neg_eids], neg_v[train_neg_eids]
val_neg_u, val_neg_v = neg_u[val_neg_eids], neg_v[val_neg_eids]

train_pos_g = dgl.graph((train_pos_u, train_pos_v), num_nodes=g.number_of_nodes())# this is just the edge_list
train_neg_g = dgl.graph((train_neg_u, train_neg_v), num_nodes=g.number_of_nodes())
test_pos_g = dgl.graph((test_pos_u, test_pos_v), num_nodes=g.number_of_nodes())
test_neg_g = dgl.graph((test_neg_u, test_neg_v), num_nodes=g.number_of_nodes())
val_pos_g = dgl.graph((val_pos_u, val_pos_v), num_nodes=g.number_of_nodes())
val_neg_g = dgl.graph((val_neg_u, val_neg_v), num_nodes=g.number_of_nodes())

print("Total number of edges: ",g_.number_of_edges())
print("num of train POSITIVE edges: ",train_pos_g.number_of_edges(),", num of test POSITIVE edges: ",test_pos_g.number_of_edges(),"num of val POSITIVE edges: ",val_pos_g.number_of_edges())
print("num of train NEGATIVE edges: ",train_neg_g.number_of_edges(),", num of test NEGATIVE edges: ",test_neg_g.number_of_edges(),"num of val NEGATIVE edges: ",val_neg_g.number_of_edges())

Total number of edges:  2525926
num of train POSITIVE edges:  306244 , num of test POSITIVE edges:  38280 num of val POSITIVE edges:  38280
num of train NEGATIVE edges:  306244 , num of test NEGATIVE edges:  38280 num of val NEGATIVE edges:  38280


In [ ]:
'''
Data prep for CASTER. This part returns negative edges
'''

neg_u, neg_v
neg_u_s=[]
neg_v_s=[]

numba.njit(target="cuda")
def func(neg_u):
  for i in neg_u:
    for j,k in D.items():
      if i==k:
        neg_u_s.append(j)
        print('o')
        break
  return neg_u_s      
function1=jit(parallel=True) (func)
neg_u_s = function1(neg_u) 

numba.njit(target="cuda")
def func(neg_v):
  for i in neg_v:
    for j,k in D.items():
      if i==k:
        neg_v_s.append(j)
        print('o')
        break
  return neg_v_s      
function1=jit(parallel=True) (func)
neg_v_s = function1(neg_v)   

D_1={}
for i in range (len(df[1])):
    D_1[df[0][i]]=df[1][i]

neg_u_s,neg_v_s
neg_u_s1=[]
neg_v_s1=[]

numba.njit(target="cuda")
def func(neg_u_s):
  for i in neg_u_s:
    for j,k in D_1.items():
      if i==j:
        neg_u_s1.append(k)
        print('o')
        break
  return neg_u_s1     
function1=jit(parallel=True) (func)
neg_u_s1 = function1(neg_u_s) 

numba.njit(target="cuda")
def func(neg_v_s):
  for i in neg_v_s:
    for j,k in D_1.items():
      if i==j:
        neg_v_s1.append(k)
        print('o')
        break
  return neg_v_s1      
function1=jit(parallel=True) (func)
neg_v_s1 = function1(neg_v_s)  

LEN=len(u)
a_list_neg=[0]*LEN
len(a_list_neg)

df_re_reg = pd.DataFrame(list(zip(neg_u_s, neg_u_s1,neg_v_s,neg_v_s1,a_list_neg)),columns =['Drug1_ID', 'Drug1_SMILES','Drug2_ID','Drug2_SMILES','label'])
df_re_reg

In [ ]:
'''
Data prep for CASTER. This part returns positive edges
'''
u,v
u_s=[]
v_s=[]

numba.njit(target="cuda")
def func(u):
  for i in u:
    for j,k in D.items():
      if i==k:
        u_s.append(j)
        print('o')
        break
  return u_s      
function1=jit(parallel=True) (func)
u_s = function1(u) 

numba.njit(target="cuda")
def func(v):
  for i in v:
    for j,k in D.items():
      if i==k:
        v_s.append(j)
        print('o')
        break
  return v_s      
function1=jit(parallel=True) (func)
v_s = function1(v)   

D_1={}
for i in range (len(df[1])):
    D_1[df[0][i]]=df[1][i]

u_s,v_s
u_s1=[]
v_s1=[]

numba.njit(target="cuda")
def func(u_s):
  for i in u_s:
    for j,k in D_1.items():
      if i==j:
        u_s1.append(k)
        print('o')
        break
  return u_s1     
function1=jit(parallel=True) (func)
u_s1 = function1(u_s) 

numba.njit(target="cuda")
def func(v_s):
  for i in v_s:
    for j,k in D_1.items():
      if i==j:
        v_s1.append(k)
        print('o')
        break
  return v_s1      
function1=jit(parallel=True) (func)
v_s1 = function1(v_s)  

LEN=len(u)
a_list_pos=[1]*LEN
len(a_list_pos)

In [ ]:
df_re = pd.DataFrame(list(zip(u_s, u_s1,v_s,v_s1,a_list_pos)),columns =['Drug1_ID', 'Drug1_SMILES','Drug2_ID','Drug2_SMILES','label'])
print(len(df_re_reg))
df_re = df_re.append(df_re_reg, ignore_index=True)
df_re = df_re.sample(frac=1).reset_index(drop=True)
df_re.to_csv('/content/drive/MyDrive/My Drive/Colab Notebooks/Hypergraph_my/data/TDC_DrugBank_FOR_CASTER.csv',index=None)

In [13]:
print("Total number of edges: ",g.number_of_edges())
print("num of train POSITIVE edges: ",train_pos_g.number_of_edges(),", num of test POSITIVE edges: ",test_pos_g.number_of_edges(),"num of val POSITIVE edges: ",val_pos_g.number_of_edges())
print("num of train NEGATIVE edges: ",train_neg_g.number_of_edges(),", num of test NEGATIVE edges: ",test_neg_g.number_of_edges(),"num of val NEGATIVE edges: ",val_neg_g.number_of_edges())

g_for_baseline = dgl.remove_edges(g, test_eids)
#dgl.save_graphs("drive/My Drive/Colab Notebooks/Hypergraph_my/a.bin",g_for_baseline)
g_for_baseline = dgl.add_self_loop(g_for_baseline)

Total number of edges:  126946


NameError: ignored

In [ ]:
def compute_loss(pos_score, neg_score):
    scores = torch.cat([pos_score, neg_score])
    labels = torch.cat([torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])])
    return F.binary_cross_entropy_with_logits(scores, labels)

def compute_auc(pos_score, neg_score):
    scores = torch.cat([pos_score, neg_score]).numpy()
    labels = torch.cat([torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])]).numpy() 
    precision, recall, thresholds = precision_recall_curve(labels, scores)
    auc_precision_recall = auc(recall, precision)
    return roc_auc_score(labels, scores),auc(recall, precision)

In [ ]:
import dgl.function as fn
class DotPredictor(nn.Module):
    def forward(self, g, h):
        with g.local_scope():
            g.ndata['h'] = h # here h is 822 drug features and g is the pos/neg train/test graph which is nothing but edge lis
            g.apply_edges(fn.u_dot_v('h', 'h', 'score'))
            return g.edata['score'][:, 0]

class MLPPredictor(nn.Module):
    def __init__(self, h_feats):
        super().__init__()
        self.W1 = nn.Linear(h_feats * 2, h_feats)
        self.W2 = nn.Linear(h_feats, 1)

    def apply_edges(self, edges):
        h = torch.cat([edges.src['h'], edges.dst['h']], 1)
        return {'score': self.W2(F.relu(self.W1(h))).squeeze(1)}

    def forward(self, g, h):
        with g.local_scope():
            g.ndata['h'] = h
            g.apply_edges(self.apply_edges)
            return g.edata['score']


decoder = MLPPredictor(128)
#decoder = DotPredictor()# You can replace DotPredictor with MLPPredictor.
#opt = torch.optim.Adam(list(model.parameters()) + list(pred.parameters()))

In [ ]:
class HyGNN(nn.Module):
    # edge attention  version
    def __init__(self, input_dim, query_dim, vertex_dim, edge_dim, dropout):
        super(HyGNN, self).__init__()
        self.dropout = dropout
        
        self.query_dim = query_dim
        self.vtx_lin_1layer = torch.nn.Linear(input_dim, vertex_dim)
        self.vtx_lin = torch.nn.Linear(vertex_dim, vertex_dim)
        
        self.qe_lin = torch.nn.Linear(edge_dim, query_dim)
        self.kv_lin = torch.nn.Linear(vertex_dim, query_dim)
        self.vv_lin = torch.nn.Linear(vertex_dim, edge_dim)
        
        self.qv_lin = torch.nn.Linear(vertex_dim, query_dim)
        self.ke_lin = torch.nn.Linear(edge_dim, query_dim)
        self.ve_lin = torch.nn.Linear(edge_dim, vertex_dim)
        
      
    def attention(self, edges):
        attn_score = F.leaky_relu((edges.src['k'] * edges.dst['q']).sum(-1))
        return {'Attn': attn_score/np.sqrt(self.query_dim)}
    
    def message_func(self, edges):
        return {'v': edges.src['v'], 'Attn': edges.data['Attn']}

    def reduce_func(self, nodes):
        attention_score = F.softmax((nodes.mailbox['Attn']), dim=1)
        aggr = torch.sum(attention_score.unsqueeze(-1) * nodes.mailbox['v'], dim=1)
        return {'h': aggr}

    def forward(self, hyG, vfeat, efeat, first_layer, last_layer):
            if first_layer:
                feat_e = self.vtx_lin_1layer(efeat)
            else:
                feat_e = self.vtx_lin(efeat)        
            feat_v = vfeat

            hyG.ndata['h'] = {'edge': feat_e}
            hyG.ndata['k'] = {'edge' : self.ke_lin(feat_e)}
            hyG.ndata['v'] = {'edge' : self.ve_lin(feat_e)}
            hyG.ndata['q'] = {'node' : self.qv_lin(feat_v)}
            hyG.apply_edges(self.attention, etype='con')
            hyG.update_all(self.message_func, self.reduce_func, etype='con')
            feat_v = hyG.ndata['h']['node']
            hyG.ndata['k'] = {'node' : self.kv_lin(feat_v)}
            hyG.ndata['v'] = {'node' : self.vv_lin(feat_v)}
            hyG.ndata['q'] = {'edge' : self.qe_lin(feat_e)}
            hyG.apply_edges(self.attention, etype='in')
            hyG.update_all(self.message_func, self.reduce_func, etype='in')
            feat_e = hyG.ndata['h']['edge']
            
            if not last_layer :
                feat_v = F.dropout(feat_v, self.dropout)
            if last_layer:
              
                return feat_v, feat_e
            else:
                return [hyG, feat_v, feat_e]

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.gat1 = HyGNN(drug_X.shape[1],64,128,128,0.5)
 
    def forward(self,hyG, v_feat, e_feat,f,l):   
        h = self.gat1(hyG, v_feat, e_feat,f,l)
        return h
model = Model()  

In [ ]:
# ----------- 3. set up loss and optimizer -------------- #
optimizer = torch.optim.Adam(itertools.chain(model.parameters(), decoder.parameters()), lr=0.005)
best_val_loss=1e10
patience=0

for e in range(500):
    # forward
    model.train()
    h=model(hyG, v_feat, e_feat,True,True)
    h=h[1]
    pos_score = decoder(train_pos_g, h)
    neg_score = decoder(train_neg_g, h)
    loss = compute_loss(pos_score, neg_score)
    
    # backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    with torch.no_grad():
      model.eval()
      pos_score = decoder(val_pos_g, h)
      neg_score = decoder(val_neg_g, h)
      #val_acc=compute_auc(pos_score, neg_score)
      val_loss = compute_loss(pos_score, neg_score)
      if val_loss<best_val_loss:
        best_val_loss=val_loss
        H=h
        E=e
        patience=0
        torch.save(decoder.state_dict(),'latest.pth')
      else:
        patience+=1  
    if patience>200:
      break

    if e % 10 == 0:
      print('In epoch {}, train loss: {:.4f}, val loss: {:.4f} (best val loss: {:.4f})'.format(e, loss, val_loss, best_val_loss))

In [ ]:
decoder.load_state_dict(torch.load('latest.pth'))
with torch.no_grad():
  model.eval()
  
  pos_score = decoder(test_pos_g, H)
  neg_score = decoder(test_neg_g, H)
  test_acc=compute_auc(pos_score, neg_score)

scores = torch.cat([pos_score, neg_score])
labels = torch.cat([torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])])

m1 = tf.keras.metrics.BinaryAccuracy()
m1.update_state(labels,scores)

sig_scores=F.sigmoid(scores)
m2 = tf.keras.metrics.Precision()
m2.update_state(labels,sig_scores)
M2=m2.result().numpy()

m3 = tf.keras.metrics.Recall()
m3.update_state(labels,sig_scores)
M3=m3.result().numpy()

F1=2*(M2*M3)/(M2+M3)
print(' Best Epoch: {}, Accuracy: {:.4f}, Precision: {:.4f}, Recall: {:.4f}, F1-score {:.4f}, ROC-AUC {:.4f}, PR-AUC {:.4f}'.format( E,m1.result().numpy(), M2, M3, F1,test_acc[0],test_acc[1]))

In [ ]:
################################################## Case study #########################################3

In [ ]:
lst=[]
for i in range (25388,26410): # range s ur choice 
  if sig_scores[i]<=0.0000001: # use a large threshold say 0.9
    #print(sig_scores[i])
    lst.append(i)
print(len(lst))
sig_scores[lst[0]]

In [ ]:
c=0
for i in lst:
  j=i-25388
  print(sig_scores[i],test_neg_u[j],test_neg_v[j])
  for m,n in D.items():
    if n==test_neg_v[j]:
      #print(m)
      break
  c+=1

In [ ]:
for i,j in D.items():
  if i=='CID000003198':
    print(j)

493


In [ ]:
test_neg_g.has_edge_between(363,290)

True

In [ ]:
# where we are cross-checking between what we predicted and what is in the DrugBank (label)
for i in range (len(mat)):
  for j in range (len(mat)):
    if i==493  and j==639:
      print(mat[i][j])

0.0


In [ ]:
for i in range(len(test_neg_u)):
  if test_neg_u[i]==126 and test_neg_v[i]==338:
    print(i)

In [ ]:
for i in range (len(test_pos_u)):
  if test_pos_u[i]==656 and test_pos_v[i]==622:
    print(sig_scores[i])
  if test_pos_u[i]==622 and test_pos_v[i]==656:
    print(sig_scores[i])  

In [ ]:
********************************************************** GCN conv with ML **********************************

In [ ]:
'''
*************another approach
Train, test (both positive and negative) graphs creation for training and testing purpose from the dgl graph (g)
Source: DGL
'''
np.random.seed(42)
# Split edge set for training and testing
u, v = g.edges()
eids = np.arange(g.number_of_edges())
eids = np.random.permutation(eids)
eids=eids.tolist()
test_size = int(len(eids) * 0.8)
train_size=len(eids)-(test_size)
test_size1=int(test_size/2)


T1=[]
T2=[]
numba.njit(target="cuda")
def func2(T1):
  for i in eids:
    m, n = int(u[i]), int(v[i])
    c=(m,n)
    T1.append(c)
  return T1  
function1=jit(parallel=True) (func2)
T1 = function1(T1)


numba.njit(target="cuda")
def func3(T1,T2):
  for i in T1:
    m=i[0]
    n=i[1]
    c=(m,n)
    d=(n,m)
    if c not in T2 and d not in T2:
      T2.append(c)
    if len(T2)>=test_size1:
      break  
  return T2
function1=jit(parallel=True) (func3)
T2 = function1(T1,T2)


test_eids=[]
numba.njit(target="cuda")
def func4(T2,test_eids,g):
  for i in T2:
    m= i[0]
    n=i[1]
    c=g.edge_id(m,n)
    d=g.edge_id(n,m)
    test_eids.append(c)
    test_eids.append(d)
  return test_eids  
function1=jit(parallel=True) (func4)
test_eids = function1(T2,test_eids,g)


train_eids=[]
numba.njit(target="cuda")
def func6(eids,test_eids,train_eids):
  i=0
  for j in eids:
    a=int(eids[i])
    if a not in test_eids  and len(train_eids)<train_size:
      train_eids.append(a)  
    i=i+1
  return train_eids    
function1=jit(parallel=True) (func6)
train_eids = function1(eids,test_eids,train_eids)     

# main one. splitting into train and test set.
test_pos_u, test_pos_v = u[test_eids], v[test_eids]
train_pos_u, train_pos_v = u[train_eids], v[train_eids]



# Find all negative edges and split them for training and testing
adj = sp.coo_matrix((np.ones(len(u)), (u.numpy(), v.numpy())))
adj_neg = 1 - adj.todense() - np.eye(g.number_of_nodes())
neg_u, neg_v = np.where(adj_neg != 0)

neg_eids = np.random.choice(len(neg_u), g.number_of_edges(),replace=False)
g_=nx.from_numpy_matrix(adj_neg)
g_=dgl.from_networkx(g_)

T1=[]
T2=[]
numba.njit(target="cuda")
def func2(T1):
  for i in neg_eids:
    m, n = int(neg_u[i]), int(neg_v[i])
    c=(m,n)
    T1.append(c)
  return T1  
function1=jit(parallel=True) (func2)
T1 = function1(T1)


numba.njit(target="cuda")
def func3(T1,T2):
  for i in T1:
    m=i[0]
    n=i[1]
    c=(m,n)
    d=(n,m)
    if c not in T2 and d not in T2:
      T2.append(c)
    if len(T2)>=test_size1:
      break  
  return T2
function1=jit(parallel=True) (func3)
T2 = function1(T1,T2)


test_neg_eids=[]
numba.njit(target="cuda")
def func4(T2,test_eids,g):
  for i in T2:
    m= i[0]
    n=i[1]
    c=g.edge_id(m,n)
    d=g.edge_id(n,m)
    test_neg_eids.append(c)
    test_neg_eids.append(d)
  return test_neg_eids  
function1=jit(parallel=True) (func4)
test_neg_eids = function1(T2,test_neg_eids,g_)


train_neg_eids=[]
numba.njit(target="cuda")
def func6(neg_eids,test_neg_eids,train_neg_eids):
  i=0
  for j in neg_eids:
    a=int(neg_eids[i])
    if a not in test_neg_eids and len(train_neg_eids)<(train_size):
      train_neg_eids.append(a)  
    i=i+1
  return train_neg_eids    
function1=jit(parallel=True) (func6)
train_neg_eids = function1(neg_eids,test_neg_eids,train_neg_eids)     

test_neg_u, test_neg_v = neg_u[test_neg_eids], neg_v[test_neg_eids]
train_neg_u, train_neg_v = neg_u[train_neg_eids], neg_v[train_neg_eids]


train_pos_g = dgl.graph((train_pos_u, train_pos_v), num_nodes=g.number_of_nodes())# this is just the edge_list
train_neg_g = dgl.graph((train_neg_u, train_neg_v), num_nodes=g.number_of_nodes())
test_pos_g = dgl.graph((test_pos_u, test_pos_v), num_nodes=g.number_of_nodes())
test_neg_g = dgl.graph((test_neg_u, test_neg_v), num_nodes=g.number_of_nodes())

print("Total number of edges: ",g.number_of_edges())
print("num of train POSITIVE edges: ",train_pos_g.number_of_edges(),", num of test POSITIVE edges: ",test_pos_g.number_of_edges())
print("num of train NEGATIVE edges: ",train_neg_g.number_of_edges(),", num of test NEGATIVE edges: ",test_neg_g.number_of_edges())

Total number of edges:  382804
num of train POSITIVE edges:  76561 , num of test POSITIVE edges:  306242
num of train NEGATIVE edges:  76561 , num of test NEGATIVE edges:  306242


In [ ]:
g_for_baseline = dgl.remove_edges(g, test_eids)
#dgl.save_graphs("drive/My Drive/Colab Notebooks/Hypergraph_my/a.bin",g_for_baseline)
g_for_baseline = dgl.add_self_loop(g_for_baseline)

In [ ]:
class MLPPredictor(nn.Module):
    def __init__(self, h_feats):
        super().__init__()
    def apply_edges(self, edges):
        h = torch.cat([edges.src['h'], edges.dst['h']], 1).squeeze(1)
        return {'score': h.squeeze(1)}

    def forward(self, g, h):
        with g.local_scope():
            g.ndata['h'] = h
            g.apply_edges(self.apply_edges)
            return g.edata['score']

decoder = MLPPredictor(128)

In [ ]:
from dgl.nn import GraphConv
# ----------- 2. create model -------------- #
# build a two-layer GraphSAGE model
class GCN(nn.Module):
    def __init__(self, in_feats, h_feats):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, h_feats)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

model = GCN(LEN, 128)

In [ ]:
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline


In [ ]:
e_feat=torch.zeros(LEN,LEN)
e_feat.fill_diagonal_(1)

tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 0., 1.]])

In [ ]:
'''
when u want to use saved GCN embeddings
'''
h =np.loadtxt("/content/drive/MyDrive/My Drive/Colab Notebooks/Hypergraph_my/data/afterTrainingEmbedsGAT.txt")
h=torch.tensor(h)
h=h.double()

In [ ]:
h = model(g_for_baseline, e_feat)
pos_score_train = decoder(train_pos_g, h)
neg_score_train = decoder(train_neg_g, h)
pos_score_test = decoder(test_pos_g, h)
neg_score_test = decoder(test_neg_g, h)

scores_train = torch.cat([pos_score_train, neg_score_train]).detach().numpy()
labels_train = torch.cat([torch.ones(pos_score_train.shape[0]), torch.zeros(neg_score_train.shape[0])]).detach().numpy()
scores_test = torch.cat([pos_score_test, neg_score_test]).detach().numpy()
labels_test = torch.cat([torch.ones(pos_score_test.shape[0]), torch.zeros(neg_score_test.shape[0])]).detach().numpy()

In [ ]:
print("LR")
lr = LogisticRegression(class_weight="balanced")
lr.fit(scores_train, labels_train)
predictions = lr.predict(scores_test)
precision, recall, thresholds = precision_recall_curve(labels_test, predictions)
auc_precision_recall = auc(recall, precision)
print("accuracy: ",accuracy_score(labels_test,predictions),"precision: ",precision_score(labels_test,predictions,average='weighted'),'recall: ',recall_score(labels_test,predictions,average='weighted'),'f1-score: ',f1_score(labels_test,predictions,average='weighted'),'ROC-AUC: ',roc_auc_score(labels_test, predictions),'PR-AUC: ',auc_precision_recall)

print("NB")
gnb = GaussianNB()
gnb.fit(scores_train, labels_train)
predictions = gnb.predict(scores_test)
precision, recall, thresholds = precision_recall_curve(labels_test, predictions)
auc_precision_recall = auc(recall, precision)
print("accuracy: ",accuracy_score(labels_test,predictions),"precision: ",precision_score(labels_test,predictions,average='weighted'),'recall: ',recall_score(labels_test,predictions,average='weighted'),'f1-score: ',f1_score(labels_test,predictions,average='weighted'),'ROC-AUC: ',roc_auc_score(labels_test, predictions),'PR-AUC: ',auc_precision_recall)

print("RF")
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(scores_train, labels_train)
predictions = clf.predict(scores_test)
precision, recall, thresholds = precision_recall_curve(labels_test, predictions)
auc_precision_recall = auc(recall, precision)
print("accuracy: ",accuracy_score(labels_test,predictions),"precision: ",precision_score(labels_test,predictions,average='weighted'),'recall: ',recall_score(labels_test,predictions,average='weighted'),'f1-score: ',f1_score(labels_test,predictions,average='weighted'),'ROC-AUC: ',roc_auc_score(labels_test, predictions),'PR-AUC: ',auc_precision_recall)


In [ ]:
********************************************************* Graph SAGE with ML ********************************************8

In [ ]:
from dgl.nn import SAGEConv

# ----------- 2. create model -------------- #
# build a two-layer GraphSAGE model
class GraphSAGE(nn.Module):
    def __init__(self, in_feats, h_feats):
        super(GraphSAGE, self).__init__()
        self.conv1 = SAGEConv(in_feats, h_feats, 'pool')
        self.conv2 = SAGEConv(h_feats, h_feats, 'pool')

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

model = GraphSAGE(1706, 128)

In [ ]:
'''
when u want to use saved GCN embeddings
'''
h =np.loadtxt("/content/drive/MyDrive/My Drive/Colab Notebooks/Hypergraph_my/data/afterTrainingEmbedsGS_DB.txt")
h=torch.tensor(h)
h=h.double()

In [ ]:
#h = model(g_for_baseline, e_feat)
pos_score_train = decoder(train_pos_g, h)
neg_score_train = decoder(train_neg_g, h)
pos_score_test = decoder(test_pos_g, h)
neg_score_test = decoder(test_neg_g, h)

scores_train = torch.cat([pos_score_train, neg_score_train]).detach().numpy()
labels_train = torch.cat([torch.ones(pos_score_train.shape[0]), torch.zeros(neg_score_train.shape[0])]).detach().numpy()
scores_test = torch.cat([pos_score_test, neg_score_test]).detach().numpy()
labels_test = torch.cat([torch.ones(pos_score_test.shape[0]), torch.zeros(neg_score_test.shape[0])]).detach().numpy()

In [ ]:
print("LR")
lr = LogisticRegression(class_weight="balanced")
lr.fit(scores_train, labels_train)
predictions = lr.predict(scores_test)
precision, recall, thresholds = precision_recall_curve(labels_test, predictions)
auc_precision_recall = auc(recall, precision)
print("accuracy: ",accuracy_score(labels_test,predictions),"precision: ",precision_score(labels_test,predictions,average='weighted'),'recall: ',recall_score(labels_test,predictions,average='weighted'),'f1-score: ',f1_score(labels_test,predictions,average='weighted'),'ROC-AUC: ',roc_auc_score(labels_test, predictions),'PR-AUC: ',auc_precision_recall)

print("NB")
gnb = GaussianNB()
gnb.fit(scores_train, labels_train)
predictions = gnb.predict(scores_test)
precision, recall, thresholds = precision_recall_curve(labels_test, predictions)
auc_precision_recall = auc(recall, precision)
print("accuracy: ",accuracy_score(labels_test,predictions),"precision: ",precision_score(labels_test,predictions,average='weighted'),'recall: ',recall_score(labels_test,predictions,average='weighted'),'f1-score: ',f1_score(labels_test,predictions,average='weighted'),'ROC-AUC: ',roc_auc_score(labels_test, predictions),'PR-AUC: ',auc_precision_recall)

print("RF")
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(scores_train, labels_train)
predictions = clf.predict(scores_test)
precision, recall, thresholds = precision_recall_curve(labels_test, predictions)
auc_precision_recall = auc(recall, precision)
print("accuracy: ",accuracy_score(labels_test,predictions),"precision: ",precision_score(labels_test,predictions,average='weighted'),'recall: ',recall_score(labels_test,predictions,average='weighted'),'f1-score: ',f1_score(labels_test,predictions,average='weighted'),'ROC-AUC: ',roc_auc_score(labels_test, predictions),'PR-AUC: ',auc_precision_recall)


LR
accuracy:  0.5771987513143201 precision:  0.577202227668201 recall:  0.5771987513143201 f1-score:  0.5771939916611587 ROC-AUC:  0.57719875131432 PR-AUC:  0.6834805709662735
NB
accuracy:  0.5452273038969181 precision:  0.5455359594068304 recall:  0.5452273038969181 f1-score:  0.54445535153448 ROC-AUC:  0.545227303896918 PR-AUC:  0.6674916000028444
RF
accuracy:  0.5882814898021825 precision:  0.5916279151800284 recall:  0.5882814898021825 f1-score:  0.5844876650167337 ROC-AUC:  0.5882814898021825 PR-AUC:  0.7080173396605481


In [ ]:
******************************************************************** GAT with ML ******************************************

In [ ]:
from dgl.nn import GATConv

# ----------- 2. create model -------------- #
# build a two-layer GraphSAGE model
class GAT(nn.Module):
    def __init__(self, in_feats, h_feats):
        super(GAT, self).__init__()
        self.conv1 = GATConv(in_feats, h_feats, num_heads=1)
        self.conv2 = GATConv(h_feats, h_feats,num_heads=1)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

model = GAT(LEN, 128)

In [ ]:
h = model(g_for_baseline, e_feat)
h=h.reshape(LEN,128)
pos_score_train = decoder(train_pos_g, h)
neg_score_train = decoder(train_neg_g, h)
pos_score_test = decoder(test_pos_g, h)
neg_score_test = decoder(test_neg_g, h)

scores_train = torch.cat([pos_score_train, neg_score_train]).detach().numpy()
labels_train = torch.cat([torch.ones(pos_score_train.shape[0]), torch.zeros(neg_score_train.shape[0])]).detach().numpy()
scores_test = torch.cat([pos_score_test, neg_score_test]).detach().numpy()
labels_test = torch.cat([torch.ones(pos_score_test.shape[0]), torch.zeros(neg_score_test.shape[0])]).detach().numpy()

In [ ]:
print("LR")
lr = LogisticRegression(class_weight="balanced")
lr.fit(scores_train, labels_train)
predictions = lr.predict(scores_test)
precision, recall, thresholds = precision_recall_curve(labels_test, predictions)
auc_precision_recall = auc(recall, precision)
print("accuracy: ",accuracy_score(labels_test,predictions),"precision: ",precision_score(labels_test,predictions,average='weighted'),'recall: ',recall_score(labels_test,predictions,average='weighted'),'f1-score: ',f1_score(labels_test,predictions,average='weighted'),'ROC-AUC: ',roc_auc_score(labels_test, predictions),'PR-AUC: ',auc_precision_recall)

print("NB")
gnb = GaussianNB()
gnb.fit(scores_train, labels_train)
predictions = gnb.predict(scores_test)
precision, recall, thresholds = precision_recall_curve(labels_test, predictions)
auc_precision_recall = auc(recall, precision)
print("accuracy: ",accuracy_score(labels_test,predictions),"precision: ",precision_score(labels_test,predictions,average='weighted'),'recall: ',recall_score(labels_test,predictions,average='weighted'),'f1-score: ',f1_score(labels_test,predictions,average='weighted'),'ROC-AUC: ',roc_auc_score(labels_test, predictions),'PR-AUC: ',auc_precision_recall)

print("RF")
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(scores_train, labels_train)
predictions = clf.predict(scores_test)
precision, recall, thresholds = precision_recall_curve(labels_test, predictions)
auc_precision_recall = auc(recall, precision)
print("accuracy: ",accuracy_score(labels_test,predictions),"precision: ",precision_score(labels_test,predictions,average='weighted'),'recall: ',recall_score(labels_test,predictions,average='weighted'),'f1-score: ',f1_score(labels_test,predictions,average='weighted'),'ROC-AUC: ',roc_auc_score(labels_test, predictions),'PR-AUC: ',auc_precision_recall)


LR
accuracy:  0.690282131661442 precision:  0.7055220776873141 recall:  0.690282131661442 f1-score:  0.6844321060518024 ROC-AUC:  0.6902821316614419 PR-AUC:  0.7813874477019046
NB
accuracy:  0.6459770114942529 precision:  0.7669291589492024 recall:  0.6459770114942529 f1-score:  0.6007494588816885 ROC-AUC:  0.6459770114942529 PR-AUC:  0.7892609409191801
RF
accuracy:  0.7202455590386625 precision:  0.7472169511055602 recall:  0.7202455590386625 f1-score:  0.7124012978347476 ROC-AUC:  0.7202455590386626 PR-AUC:  0.8041294900541032


In [ ]:
***************************************************************** DeepWalk **********************************

In [ ]:
g_for_baseline_adj_matrix=g_for_baseline.adjacency_matrix_scipy()
G_baseline = nx.from_numpy_array(g_for_baseline_adj_matrix,create_using=nx.DiGraph)
nx.write_edgelist(G_baseline, "/content/drive/MyDrive/My Drive/Colab Notebooks/Hypergraph_my/data/G_baseline_DB.txt", data=False)

# train model and generate embedding
model = DeepWalk(walk_length=100, dimensions=128, window_size=5)
model.fit(G_baseline)
h = model.get_embedding()
h=torch.from_numpy(h)

In [ ]:
pos_score_train = decoder(train_pos_g, h)
neg_score_train = decoder(train_neg_g, h)
pos_score_test = decoder(test_pos_g, h)
neg_score_test = decoder(test_neg_g, h)

scores_train = torch.cat([pos_score_train, neg_score_train]).detach().numpy()
labels_train = torch.cat([torch.ones(pos_score_train.shape[0]), torch.zeros(neg_score_train.shape[0])]).detach().numpy()
scores_test = torch.cat([pos_score_test, neg_score_test]).detach().numpy()
labels_test = torch.cat([torch.ones(pos_score_test.shape[0]), torch.zeros(neg_score_test.shape[0])]).detach().numpy()

In [ ]:
print("LR")
lr = LogisticRegression(class_weight="balanced")
lr.fit(scores_train, labels_train)
predictions = lr.predict(scores_test)
precision, recall, thresholds = precision_recall_curve(labels_test, predictions)
auc_precision_recall = auc(recall, precision)
print("accuracy: ",accuracy_score(labels_test,predictions),"precision: ",precision_score(labels_test,predictions,average='weighted'),'recall: ',recall_score(labels_test,predictions,average='weighted'),'f1-score: ',f1_score(labels_test,predictions,average='weighted'),'ROC-AUC: ',roc_auc_score(labels_test, predictions),'PR-AUC: ',auc_precision_recall)

print("NB")
gnb = GaussianNB()
gnb.fit(scores_train, labels_train)
predictions = gnb.predict(scores_test)
precision, recall, thresholds = precision_recall_curve(labels_test, predictions)
auc_precision_recall = auc(recall, precision)
print("accuracy: ",accuracy_score(labels_test,predictions),"precision: ",precision_score(labels_test,predictions,average='weighted'),'recall: ',recall_score(labels_test,predictions,average='weighted'),'f1-score: ',f1_score(labels_test,predictions,average='weighted'),'ROC-AUC: ',roc_auc_score(labels_test, predictions),'PR-AUC: ',auc_precision_recall)

print("RF")
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(scores_train, labels_train)
predictions = clf.predict(scores_test)
precision, recall, thresholds = precision_recall_curve(labels_test, predictions)
auc_precision_recall = auc(recall, precision)
print("accuracy: ",accuracy_score(labels_test,predictions),"precision: ",precision_score(labels_test,predictions,average='weighted'),'recall: ',recall_score(labels_test,predictions,average='weighted'),'f1-score: ',f1_score(labels_test,predictions,average='weighted'),'ROC-AUC: ',roc_auc_score(labels_test, predictions),'PR-AUC: ',auc_precision_recall)


LR
accuracy:  0.7335292580982237 precision:  0.7338144849239618 recall:  0.7335292580982237 f1-score:  0.733447967379448 ROC-AUC:  0.7335292580982237 PR-AUC:  0.8005722115536301
NB
accuracy:  0.7077194357366771 precision:  0.7081833293534113 recall:  0.7077194357366771 f1-score:  0.7075565232420205 ROC-AUC:  0.7077194357366771 PR-AUC:  0.78200849753797
RF
accuracy:  0.7378657262277952 precision:  0.737966308416069 recall:  0.7378657262277952 f1-score:  0.7378380239603947 ROC-AUC:  0.7378657262277951 PR-AUC:  0.8035732863302465


In [ ]:
******************************************************************* Node2Vec ***********************************8

In [ ]:
h=np.loadtxt('/content/drive/MyDrive/My Drive/Colab Notebooks/Hypergraph_my/data/node_2vec_embeddings_DB.txt',delimiter=',')
h=torch.from_numpy(h)

In [ ]:
pos_score_train = decoder(train_pos_g, h)
neg_score_train = decoder(train_neg_g, h)
pos_score_test = decoder(test_pos_g, h)
neg_score_test = decoder(test_neg_g, h)

scores_train = torch.cat([pos_score_train, neg_score_train]).detach().numpy()
labels_train = torch.cat([torch.ones(pos_score_train.shape[0]), torch.zeros(neg_score_train.shape[0])]).detach().numpy()
scores_test = torch.cat([pos_score_test, neg_score_test]).detach().numpy()
labels_test = torch.cat([torch.ones(pos_score_test.shape[0]), torch.zeros(neg_score_test.shape[0])]).detach().numpy()

In [ ]:
print("LR")
lr = LogisticRegression(class_weight="balanced")
lr.fit(scores_train, labels_train)
predictions = lr.predict(scores_test)
precision, recall, thresholds = precision_recall_curve(labels_test, predictions)
auc_precision_recall = auc(recall, precision)
print("accuracy: ",accuracy_score(labels_test,predictions),"precision: ",precision_score(labels_test,predictions,average='weighted'),'recall: ',recall_score(labels_test,predictions,average='weighted'),'f1-score: ',f1_score(labels_test,predictions,average='weighted'),'ROC-AUC: ',roc_auc_score(labels_test, predictions),'PR-AUC: ',auc_precision_recall)

print("NB")
gnb = GaussianNB()
gnb.fit(scores_train, labels_train)
predictions = gnb.predict(scores_test)
precision, recall, thresholds = precision_recall_curve(labels_test, predictions)
auc_precision_recall = auc(recall, precision)
print("accuracy: ",accuracy_score(labels_test,predictions),"precision: ",precision_score(labels_test,predictions,average='weighted'),'recall: ',recall_score(labels_test,predictions,average='weighted'),'f1-score: ',f1_score(labels_test,predictions,average='weighted'),'ROC-AUC: ',roc_auc_score(labels_test, predictions),'PR-AUC: ',auc_precision_recall)

print("RF")
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(scores_train, labels_train)
predictions = clf.predict(scores_test)
precision, recall, thresholds = precision_recall_curve(labels_test, predictions)
auc_precision_recall = auc(recall, precision)
print("accuracy: ",accuracy_score(labels_test,predictions),"precision: ",precision_score(labels_test,predictions,average='weighted'),'recall: ',recall_score(labels_test,predictions,average='weighted'),'f1-score: ',f1_score(labels_test,predictions,average='weighted'),'ROC-AUC: ',roc_auc_score(labels_test, predictions),'PR-AUC: ',auc_precision_recall)


LR
accuracy:  0.7941627862931929 precision:  0.7961634235308683 recall:  0.7941627862931929 f1-score:  0.7938145813887677 ROC-AUC:  0.794162786293193 PR-AUC:  0.8447253247083487
NB
accuracy:  0.723205830682924 precision:  0.7249722389428003 recall:  0.723205830682924 f1-score:  0.7226614378176941 ROC-AUC:  0.7232058306829239 PR-AUC:  0.7943964278615755
RF
accuracy:  0.708978520255223 precision:  0.7266592946543649 recall:  0.708978520255223 f1-score:  0.7031902891290899 ROC-AUC:  0.708978520255223 PR-AUC:  0.7938337538991466
